<a href="https://colab.research.google.com/github/ernesto-butto/ag2-workshop-simulation/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


- Phase 1: Scenario Generation Agents
- Phase 2: Running the Scenario with a Game Master
- Phase 3: Live Playtesting
- Phase 4: AI Feedback Agent (Optional)

Instalar AG2
https://docs.ag2.ai/docs/home/quick-start

In [8]:
!pip install ag2

Setup

In [9]:
# 1. Import our agent class
from autogen import ConversableAgent

# Surpress warnings
import warnings
warnings.filterwarnings("ignore")


In [10]:
llm_config_azure_gpt_4o = {"config_list": [{
    "model": "gpt-4o",  # This should match your Azure deployment
    "api_key": "e2cc82d152f34eb88d1c4836e237f77d",
    "base_url": "https://ai-ernestobuttoai488542268715.services.ai.azure.com",
    "api_type": "azure",
    "api_version": "2024-08-01-preview",
    "temperature": 0.8
}]}

# Phase 1: Scenario Generation Agents
🔹 **Attendees Build:**

- **(A) Scenario Creation Agent** → Generates a simulation scenario.
- **(B) Mocked User Agent** → Simulates a user providing input for scenario generation.

🔹 **Simulation I (Test Run)**

- A & B **interact automatically** → Produces a detailed scenario.
- Attendees analyze the **output** to understand if it’s coherent and engaging.

In [11]:
fin_de_conversacion_keyword = "FIN DE CREACION DE ESCENARIO"

In [12]:
def createScenarioCreationAgent():

  return ConversableAgent(
      name="scenario_creation_agent",
      llm_config=llm_config_azure_gpt_4o,
      code_execution_config=False,
      human_input_mode="NEVER",
      system_message=f"""
      Eres un creador de escenarios para simulaciones conversacionales 1 a 1.
      Tu objetivo es ayudar a los estudiantes a crear escenarios de juegos de roles para practicar habilidades basado en sus hobbies e intereses.
      Explicale tu objetivo al estudiante que te salude, y hazle preguntas sobre:

      1. Que habilidades quiere practicar
      2. Algún interes o hobby para crear un escenario que pueda relacionar

      Algunos puntos a tomar en cuenta:
      - El escenario debe de ser cortos, y con un objetivo claro, que para lograrlo debe conversar con otra persona.
      - Una vez tengas la información necesaria, genera un escenario y preguntale al estudiante si está deacuerdo
      - Itera con el usuario, cuando se llegue a un escenario que le guste, escribe todo el escenario, terminado la conversación con el término: {fin_de_conversacion_keyword}
      - Nunca empiezes el escenario, siempre termina la conversación con {fin_de_conversacion_keyword} cuando el escenario esté listo.

      """,
      is_termination_msg=lambda msg: fin_de_conversacion_keyword in msg["content"],
  )


In [13]:
def mockedUserAgent(skill, hobby):
  return ConversableAgent(
        "mocked_student",
        llm_config=llm_config_azure_gpt_4o,
        human_input_mode="NEVER",  # always ask for human input
        system_message=f"You are a student learning {skill}, and you like {hobby}",
        code_execution_config=False,
        is_termination_msg=lambda msg: fin_de_conversacion_keyword in msg["content"],
    )

In [14]:
creador = createScenarioCreationAgent()
estudiante = mockedUserAgent("Negotiation", "90's pop music")
result = estudiante.initiate_chat(
    creador,
    message="Hola!",
    max_turns=12,
    summary_method="reflection_with_llm",
    summary_prompt="Escribe el escenario en Español"
)


mocked_student (to scenario_creation_agent):

Hola!

--------------------------------------------------------------------------------
scenario_creation_agent (to mocked_student):

¡Hola! Soy un creador de escenarios para simulaciones conversacionales 1 a 1. Mi objetivo es ayudarte a practicar habilidades específicas a través de juegos de roles basados en tus intereses o hobbies. 

Para empezar, me gustaría saber:

1. ¿Qué habilidades te gustaría practicar? (Por ejemplo, hablar en público, resolver conflictos, negociar, expresar emociones, etc.)
2. ¿Tienes algún interés o hobby en el que te gustaría que basemos el escenario? (Por ejemplo, deportes, videojuegos, cine, música, viajes, etc.)

¡Cuéntame un poco más para que podamos crear algo divertido y útil! 😊

--------------------------------------------------------------------------------
mocked_student (to scenario_creation_agent):

¡Qué genial! Me encantaría practicar mis habilidades de negociación, ya que estoy aprendiendo sobre eso.

KeyboardInterrupt: 